<a href="https://colab.research.google.com/github/yuri-mrs/machine-learning/blob/main/ym_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔁 Transfer Learning com MobileNetV2 no Google Colab

## ✅ Requisitos

In [1]:
!pip install -q tensorflow

## 📁 Importações

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


## 📥 Carregamento e pré-processamento de dados

In [3]:
import tensorflow_datasets as tfds

(ds_train, ds_val), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True
)

IMG_SIZE = (160, 160)
BATCH_SIZE = 32

def format_image(image, label):
    image = tf.image.resize(image, IMG_SIZE) / 255.0
    return image, label

train_batches = ds_train.map(format_image).batch(BATCH_SIZE).prefetch(1)
val_batches = ds_val.map(format_image).batch(BATCH_SIZE).prefetch(1)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.COP8WX_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


## 🧠 Carregar o modelo pré-treinado (sem o topo)

In [4]:
base_model = MobileNetV2(input_shape=IMG_SIZE + (3,),
                         include_top=False,
                         weights='imagenet')

base_model.trainable = False  # Congela os pesos


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## 🧱 Construir o modelo com camadas adicionais

In [5]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)


## ⚙️ Compilar e treinar o modelo

In [6]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_batches,
          epochs=5,
          validation_data=val_batches)


Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 418s 710ms/step - accuracy: 0.9545 - loss: 0.1047 - val_accuracy: 0.9794 - val_loss: 0.0577
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 438s 703ms/step - accuracy: 0.9824 - loss: 0.0477 - val_accuracy: 0.9807 - val_loss: 0.0580
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 427s 733ms/step - accuracy: 0.9886 - loss: 0.0328 - val_accuracy: 0.9785 - val_loss: 0.0618
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 428s 735ms/step - accuracy: 0.9941 - loss: 0.0202 - val_accuracy: 0.9785 - val_loss: 0.0688
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 481s 826ms/step - accuracy: 0.9955 - loss: 0.0149 - val_accuracy: 0.9761 - val_loss: 0.0775


## 🧪 Fine-Tuning (opcional)

In [7]:
base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_batches,
          epochs=5,
          validation_data=val_batches)


Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 1760s 3s/step - accuracy: 0.9334 - loss: 0.1870 - val_accuracy: 0.9776 - val_loss: 0.0767
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 1746s 3s/step - accuracy: 0.9868 - loss: 0.0418 - val_accuracy: 0.9774 - val_loss: 0.0729
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 1698s 3s/step - accuracy: 0.9972 - loss: 0.0162 - val_accuracy: 0.9772 - val_loss: 0.0752
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 1701s 3s/step - accuracy: 0.9994 - loss: 0.0067 - val_accuracy: 0.9770 - val_loss: 0.0787
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 1695s 3s/step - accuracy: 0.9999 - loss: 0.0031 - val_accuracy: 0.9776 - val_loss: 0.0827


## 📌 Resumo do processo
- Transfer Learning: Utiliza um modelo pré-treinado como extrator de características.

- Congelamento da base: Evita recomeçar o aprendizado do zero.

- Camadas customizadas: Adiciona camadas específicas para o novo problema.

- Fine-tuning (opcional): Ajusta os pesos do modelo base após um pré-treinamento inicial.